In [7]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [4]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

2023-02-02 16:43:00.962517: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-02 16:43:00.963052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [46]:
for x in ratings.take(2).as_numpy_iterator():
  print(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}
{'movie_title': b'Strictly Ballroom (1992)', 'user_id': b'92'}


In [48]:
for x in movies.take(2).as_numpy_iterator():
  print(x)

b'You So Crazy (1994)'
b'Love Is All There Is (1996)'


In [5]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

movies = movies.map(lambda x: x["movie_title"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [49]:
#Splittinng the data

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [53]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_user_ids[:11]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107', b'108'], dtype=object)

In [55]:
#Implementing the Model


embedding_dimension = 32

#User model
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])


In [58]:
#Movie Model
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [59]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [63]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [65]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [77]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.1))

In [75]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [78]:
model.fit(cached_train, epochs=3)

Epoch 1/3


2023-02-07 20:41:52.174134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-07 20:41:52.723907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-07 20:41:52.747474: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 1/10 [==>...........................] - ETA: 47s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0032 - factorized_top_k/top_50_categorical_accuracy: 0.3876 - factorized_top_k/top_100_categorical_accuracy: 0.4979 - loss: 73817.4531 - regularization_loss: 0.0000e+00 - total_loss: 73817.4531

2023-02-07 20:41:55.592471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-07 20:41:55.622063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 2/10 [=====>........................] - ETA: 5s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 6.7139e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0020 - factorized_top_k/top_50_categorical_accuracy: 0.3874 - factorized_top_k/top_100_categorical_accuracy: 0.4971 - loss: 73827.0156 - regularization_loss: 0.0000e+00 - total_loss: 73827.0156

...

 8/10 [=======================>......] - ETA: 1s - factorized_top_k/top_1_categorical_accuracy: 0.0024 - factorized_top_k/top_5_categorical_accuracy: 0.0214 - factorized_top_k/top_10_categorical_accuracy: 0.0438 - factorized_top_k/top_50_categorical_accuracy: 0.7657 - factorized_top_k/top_100_categorical_accuracy: 0.8213 - loss: 67985.5488 - regularization_loss: 0.0000e+00 - total_loss: 67985.5488

2023-02-07 20:42:10.823372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-07 20:42:10.844271: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 9/10 [==========================>...] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0024 - factorized_top_k/top_5_categorical_accuracy: 0.0213 - factorized_top_k/top_10_categorical_accuracy: 0.0437 - factorized_top_k/top_50_categorical_accuracy: 0.7656 - factorized_top_k/top_100_categorical_accuracy: 0.8305 - loss: 67993.7075 - regularization_loss: 0.0000e+00 - total_loss: 67993.7075

2023-02-07 20:42:11.409524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-07 20:42:11.428790: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 6s 564ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0023 - factorized_top_k/top_5_categorical_accuracy: 0.0211 - factorized_top_k/top_10_categorical_accuracy: 0.0435 - factorized_top_k/top_50_categorical_accuracy: 0.7655 - factorized_top_k/top_100_categorical_accuracy: 0.8227 - loss: 64801.8743 - regularization_loss: 0.0000e+00 - total_loss: 64801.8743


In [79]:
#This is the converting ID's into Integers part.

user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))


movie_title_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_title_vocabulary.adapt(movies)

2023-02-07 20:42:54.393963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-07 20:43:55.850117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [81]:
#Now defining the model, by Inheriting from tfrs.Model and implement the compute_loss method

class MovieLensModel(tfrs.Model):
    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval):
        super().__init__()

        #setting up user and movie representation
        self.user_model = user_model
        self.movie_model = movie_model

        #and setting up the retrieval task
        self.task = task


        def compute_loss(self, features: Dict[Text, tf.Tensor], training:bool = False) -> tf.Tensor:
            # Here, it's about how the loss is computed
            user_embeddings = self.user_model(features["user_id"])
            movie_embeddings = self.movie_model(features["movie_title"])

            return self.task(user_embeddings, movie_embeddings)


In [42]:
MovieLensModel.compute_loss?

Signature: MovieLensModel.compute_loss(self, inputs, training: bool = False) -> tensorflow.python.framework.ops.Tensor
Docstring:
Defines the loss function.

Args:
  inputs: A data structure of tensors: raw inputs to the model. These will
    usually contain labels and weights as well as features.
  training: Whether the model is in training mode.

Returns:
  Loss tensor.
File:      ~/TheGitVerse/ClassStuffs/ClasseStuffs/lib/python3.9/site-packages/tensorflow_recommenders/models/base.py
Type:      function

In [30]:
user_ids_vocabulary.vocabulary_size(), movie_title_vocabulary.vocabulary_size()

(944, 1665)

In [82]:
#Defining the user and item/movie models,
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_title_vocabulary,
    tf.keras.layers.Embedding(movie_title_vocabulary.vocabulary_size(), 64)
])


#Defining the Objectives
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

In [88]:
#Now Time to Fit and Evaluate the model

#First, is creating a retrieval model
model = model(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam())

#Training for 3 Epochs
model.fit(ratings.batch(4096), epochs=3)


#Now using brute-force search to set up retrieval using the trained representation
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)


NotImplementedError: Exception encountered when calling layer 'model_1' (type Model).

Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.

Call arguments received by layer 'model_1' (type Model):
  • inputs=<keras.engine.sequential.Sequential object at 0x2999731f0>
  • training=<keras.engine.sequential.Sequential object at 0x1774fda00>
  • mask=<tensorflow_recommenders.tasks.retrieval.Retrieval object at 0x1774fde50>

In [85]:
#And then here we get some recommendations
_, titles = index(np.array(["42"]))
print(f"Top 3 Recommendations for user 42: {titles[0, :3]}")

NameError: name 'index' is not defined

In [11]:
tf.keras.layers.Embedding?

Init signature:
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs,
)
Docstring:     
Turns positive integers (indexes) into dense vectors of fixed size.

e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`

This layer can only be used on positive integer inputs of a fixed range. The
`tf.keras.layers.TextVectorization`, `tf.keras.layers.StringLookup`,
and `tf.keras.layers.IntegerLookup` preprocessing layers can help prepare
inputs for an `Embedding` layer.

This layer accepts `tf.Tensor` and `tf.RaggedTensor` inputs. It cannot be
called with `tf.SparseTensor` input.

Example:

>>> model = tf.keras.Sequential()
>>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
>>> # The model will take as input an integer matrix of size (batch,
>>> # input_length), and the largest integer (i.e.

In [15]:
tf.keras.Sequential?

Init signature: tf.keras.Sequential(layers=None, name=None)
Docstring:     
`Sequential` groups a linear stack of layers into a `tf.keras.Model`.

`Sequential` provides training and inference features on this model.

Examples:

```python
# Optionally, the first layer can receive an `input_shape` argument:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(16,)))
# Afterwards, we do automatic shape inference:
model.add(tf.keras.layers.Dense(4))

# This is identical to the following:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(16,)))
model.add(tf.keras.layers.Dense(8))

# Note that you can also omit the `input_shape` argument.
# In that case the model doesn't have any weights until the first call
# to a training/evaluation method (since it isn't yet built):
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
# model.weights not created yet

# Whereas if you specify the input shape, the model ge

In [5]:
tf.keras.layers.experimental.preprocessing.StringLookup?

Init signature:
tf.keras.layers.experimental.preprocessing.StringLookup(
    max_tokens=None,
    num_oov_indices=1,
    mask_token=None,
    oov_token='[UNK]',
    vocabulary=None,
    idf_weights=None,
    encoding='utf-8',
    invert=False,
    output_mode='int',
    sparse=False,
    pad_to_max_tokens=False,
    **kwargs,
)
Docstring:     
A preprocessing layer which maps string features to integer indices.

This layer translates a set of arbitrary strings into integer output via a
table-based vocabulary lookup. This layer will perform no splitting or
transformation of input strings. For a layer than can split and tokenize
natural language, see the `TextVectorization` layer.

The vocabulary for the layer must be either supplied on construction or
learned via `adapt()`. During `adapt()`, the layer will analyze a data set,
determine the frequency of individual strings tokens, and create a
vocabulary from them. If the vocabulary is capped in size, the most frequent
tokens will be used

In [92]:
len(tf.config.list_physical_devices('CPU'))

1

In [91]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
